In [ ]:
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import matplotlib.pyplot as plt
import glob

# Cargar el dataset con noticias
file_path = "scrapping/noticias_final.csv"
noticias = pd.read_csv(file_path)

# Configurar el modelo de resumen
model_name = "mrm8488/bert2bert_shared-spanish-finetuned-summarization"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

print(f"Total de noticias cargadas: {len(noticias)}")

In [4]:


def resumir_noticia(texto, max_length=130, min_length=30):
    """
    Resumir el contenido de una noticia utilizando un modelo NLP.

    Args:
        texto (str): Texto de entrada a resumir.
        max_length (int): Longitud máxima del resumen.
        min_length (int): Longitud mínima del resumen.

    Returns:
        str: Resumen generado.
    """
    texto_recortado = texto[:500]  # Limitar el texto inicial a 500 caracteres
    inputs = tokenizer.encode("summarize: " + texto_recortado, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(
        inputs, max_length=max_length, min_length=min_length, 
        length_penalty=2.0, num_beams=4, early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Resumir las noticias en bloques de 100
for i in range(0, len(noticias), 100):
    subset = noticias.iloc[i:i+100].copy()
    subset['Resumen'] = subset['Contenido'].apply(lambda x: resumir_noticia(str(x)))

    output_file = f"noticias_resumidas_{i//100 + 1}.csv"
    subset.to_csv(output_file, index=False)
    print(f"Noticias resumidas guardadas en {output_file}")

# Combinar todos los archivos CSV generados
path = "noticias_resumidas"
csv_files = glob.glob(path + "/*.csv")

# Leer y concatenar los archivos CSV
df_list = (pd.read_csv(file) for file in csv_files)
big_df = pd.concat(df_list, ignore_index=True)

# Limpiar datos duplicados y columnas irrelevantes
big_df.drop_duplicates(inplace=True)
big_df.drop(columns=['URL'], inplace=True, errors='ignore')

# Calcular el número de palabras en los resúmenes
big_df['word_count'] = big_df['Resumen'].apply(lambda x: len(str(x).split()))

# Filtrar resúmenes por longitud de palabras
big_df = big_df[(big_df['word_count'] > 10) & (big_df['word_count'] < 45)]

# Guardar el archivo final
final_output_file = "noticias_res_full.csv"
big_df.to_csv(final_output_file, index=False)
print(f"Archivo final guardado en {final_output_file}")


In [5]:

# Crear histograma del recuento de palabras
plt.figure(figsize=(10, 6))
plt.hist(big_df['word_count'], bins=40, edgecolor='black')
plt.xlabel('Número de palabras en el resumen')
plt.ylabel('Frecuencia')
plt.title('Histograma del recuento de palabras en los resúmenes')
plt.tight_layout()
plt.show()


Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "dccuchile/bert-base-spanish-wwm-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.46.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31002
}

Config of the decoder: <class 'transformers.models.bert.modeling_bert.BertLMHeadModel'> is overwritten by shared decoder config: BertConfig {
  "_name_or_path": "dccuchile/bert-base-span

6962
